In [1]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline


C:\Users\ARJUN\anaconda3\envs\myenv\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
C:\Users\ARJUN\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def extract_text_from_pdf(file_path):
    reader = PyPDF2.PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

pdf_path = "data.pdf"  # Make sure your PDF is named 'data.pdf'
text = extract_text_from_pdf(pdf_path)
print("✅ Extracted text length:", len(text))


✅ Extracted text length: 642


In [5]:
def chunk_text(text, chunk_size=500):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunks = chunk_text(text)
print("✅ Number of chunks:", len(chunks))


✅ Number of chunks: 2


In [7]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

vectors = embedding_model.encode(chunks)
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

print("✅ Vector index created!")


C:\Users\ARJUN\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


✅ Vector index created!


In [9]:
def search_chunks(query, chunks, model, index, top_k=1):
    q_vector = model.encode([query])
    _, I = index.search(q_vector, top_k)
    return [chunks[i] for i in I[0]]


In [17]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

model_name = "deepset/roberta-base-squad2"  # PyTorch-only model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


C:\Users\ARJUN\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ARJUN\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
def ask_question(question):
    context = search_chunks(question, chunks, embedding_model, index)[0]
    result = qa_pipeline(question=question, context=context)
    return result["answer"]


In [21]:
question = "What is the main idea of the document?"  # Change this as needed
answer = ask_question(question)
print("🧠 Answer:", answer)


🧠 Answer: Earth is the only planet known to support life


In [23]:
question = "How many planets are there in Solar System?"  # Change this as needed
answer = ask_question(question)
print("🧠 Answer:", answer)


🧠 Answer: eight


In [25]:
question = "Largest planet?"  # Change this as needed
answer = ask_question(question)
print("🧠 Answer:", answer)


🧠 Answer: Jupiter
